#### Import necessary Libraries for data download and manipulation

In [1]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date
import pickle
import re
import numpy as np
import sys
sys.path.append('./scripts')

label_encoder = LabelEncoder()
from data_download import *



2023-10-08 20:30:26.713651: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-08 20:30:26.713716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-08 20:30:26.713739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-08 20:30:26.728503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 20:30:27.833694: W tensorflow/compiler/

#### Global Variables

In [2]:
labels = ['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play', 'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs', 'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers']
op_labels = ['op_'+s for s in labels]
labels_2 = ['Date', 'Time', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
labels_old = ['Date', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
teams = read_teams()
schedule_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i-schedule.html'
stats_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i.html'
stats_hist_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i/gamelog/'
yearly_stats_url_base = 'https://www.sports-reference.com/cfb/schools/%s/'
# Get list of all FBS team urls
making_predictions = True

In [3]:
yearly_stats_cols = ['Year','Conf','W_overall','L_overall','T_overall','Pct_overall','W_conf','L_conf','T_conf','Pct_conf','SRS','SOS','AP_Pre','AP_High','AP_Post','CFP_High','CFP_Final','Coach','Bowl','Notes']
hist_cols = ['Date', 'Location', 'Opponent', 'Result', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play', 'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs', 'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers'] 
offfense_cols = hist_cols[:4] + ['of_' + stat for stat in hist_cols[4:]]
defense_cols = ['de_' + stat for stat in hist_cols[4:]]
all_cols = hist_cols + defense_cols
df_cols = all_cols[4:] + ['op_' + s for s in all_cols[4:]]
#all_cols

#### Helper functions

- 'save_data' - pickle save method
- 'load_data' - pickle load method
- 'get_team_stats' - web scrape for stats
- 'get_team_schedules' - web scrape schedules
- 'get_cur_teams' - web scrape for list of current teams

In [4]:
col_test = get_team_stats(stats_url_base % ('air-force', 2023))

In [5]:
col_test = get_yearly_stats(yearly_stats_url_base, 'alabama')

#### Example of schedule request post-processing
* Here we are loading in the 2020 season data for Georgia. We get this using our request and scraping method defined above.

In [6]:
col_x = get_team_schedule('https://www.sports-reference.com/cfb/schools/georgia/2020-schedule.html')
col_x[['Team', 'Opponent']] = col_x[['Team', 'Opponent']].applymap(standardize)
col_x[['W/L']] = col_x[['W/L']].apply(label_encoder.fit_transform).astype('int32')
#col_x

/tmp/ipykernel_764185/1072618318.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  col_x[['Team', 'Opponent']] = col_x[['Team', 'Opponent']].applymap(standardize)


#### Looping functions to collect all data for a year
* We can loop over the request and scrape method in order to get all the data for each team for any given year

In [7]:
all_cols

['Date',
 'Location',
 'Opponent',
 'Result',
 'pass_completion',
 'pass_attempts',
 'pass_completion_percent',
 'pass_yards',
 'pass_td',
 'rush_attempt',
 'rush_yards',
 'avg_rush_yard',
 'rush_td',
 'num_plays',
 'total_yard',
 'yard_per_play',
 'pass_first_down',
 'rush_first_down',
 'penalty_first',
 'first_downs',
 'penalty',
 'penalty_yards',
 'lost_fumble',
 'interceptions',
 'turnovers',
 'de_pass_completion',
 'de_pass_attempts',
 'de_pass_completion_percent',
 'de_pass_yards',
 'de_pass_td',
 'de_rush_attempt',
 'de_rush_yards',
 'de_avg_rush_yard',
 'de_rush_td',
 'de_num_plays',
 'de_total_yard',
 'de_yard_per_play',
 'de_pass_first_down',
 'de_rush_first_down',
 'de_penalty_first',
 'de_first_downs',
 'de_penalty',
 'de_penalty_yards',
 'de_lost_fumble',
 'de_interceptions',
 'de_turnovers']

#### Looping functions to save all data for each year in one place
* Adding in a nother loop we can itterate over all of the years that we want to get data for and then use the other looping function to get the data.

#### Download and save all the data...
* now we sit back and let the data flow in from the automated process of web scraping.

In [8]:
#download_all_stats()
#download_all_schedules()
#download_all_stats_hist()

Starting year 2023


#### Loading in all downloaded data

In [9]:
stats_data_dict = load_stats_data(making_predictions=False)
# Contains all the stats for each team for each year of data
# Format: {Year: {'team_name': pd.DataFrame, ...}, ...}
# dataframe: 3 rows x 22 Cols - all numbers - row 1 is offense, row 2 is defense, row 3 is the diff.
# cols of dataframe: ['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play','pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs','penalty', 'penalty_yards', 'lost_fumble', 'interceptions','turnovers']
schedule_data_dict = load_stats_data(filepath = './Data/Schedules/%i_schedule.pkl', making_predictions=False)
# Contains all the schedules for each team for each year of data
# Format: {Year: {'team_name': pd.DataFrame, ...}, ...}
# dataframe: variable number of rows based on number of games that season
# cols of dataframe: ['Date', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
#hist_data_dict = load_stats_data('./Data/Stats/%i_history.pkl')


## Current Prediction code

In [18]:
#stats_data_dict = add_srs(stats_data_dict)
full_dataset = make_dataframe(stats_data_dict, schedule_data_dict, years=range(2022,2024))

2022
2023


In [19]:
print(full_dataset.shape)
missing_rows = full_dataset.loc[full_dataset.isin(['']).any(axis=1)]
print(missing_rows)

(1665, 52)
Empty DataFrame
Columns: [Team, Location, Opponent, Conference, Label, Binary_Label, pass_completion, pass_attempts, pass_completion_percent, pass_yards, pass_td, rush_attempt, rush_yards, avg_rush_yard, rush_td, num_plays, total_yard, yard_per_play, pass_first_down, rush_first_down, penalty_first, first_downs, penalty, penalty_yards, lost_fumble, interceptions, turnovers, SRS, SOS, op_pass_completion, op_pass_attempts, op_pass_completion_percent, op_pass_yards, op_pass_td, op_rush_attempt, op_rush_yards, op_avg_rush_yard, op_rush_td, op_num_plays, op_total_yard, op_yard_per_play, op_pass_first_down, op_rush_first_down, op_penalty_first, op_first_downs, op_penalty, op_penalty_yards, op_lost_fumble, op_interceptions, op_turnovers, op_SRS, op_SOS]
Index: []

[0 rows x 52 columns]


In [20]:
#full_dataset = full_dataset.drop(index=missing_rows.index)
print(full_dataset.shape)

(1665, 52)


In [21]:
full_dataset

,Team,Location,Opponent,Conference,Label,Binary_Label,pass_completion,pass_attempts,pass_completion_percent,pass_yards,...,op_rush_first_down,op_penalty_first,op_first_downs,op_penalty,op_penalty_yards,op_lost_fumble,op_interceptions,op_turnovers,op_SRS,op_SOS
0,0.0,0,23.0,8,41.0,1,3.200000,6.700000,47.099998,70.500000,...,12.5,1.2,25.799999,6.0,53.500000,0.4,0.5,0.9,-12.08,7.92
1,0.0,1,132.0,7,14.0,0,3.200000,6.700000,47.099998,70.500000,...,8.8,1.5,19.600000,5.2,48.799999,0.6,0.5,1.1,-5.59,-4.43
2,0.0,0,70.0,7,48.0,1,3.200000,6.700000,47.099998,70.500000,...,7.8,1.9,19.600000,8.4,72.500000,0.7,0.9,1.6,-15.62,-5.04
3,0.0,0,68.0,1,13.0,1,3.200000,6.700000,47.099998,70.500000,...,5.3,1.1,15.600000,5.7,45.500000,0.8,0.7,1.5,-2.56,0.94
4,0.0,1,119.0,7,27.0,0,3.200000,6.700000,47.099998,70.500000,...,10.0,2.2,21.400000,6.9,62.000000,0.5,0.8,1.4,-9.52,-4.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,130.0,1,102.0,0,7.0,0,18.000000,31.200001,57.700001,200.800003,...,5.6,3.2,17.799999,6.8,67.000000,1.0,1.2,2.2,9.62,-2.18
1661,130.0,1,42.0,3,10.0,0,18.000000,31.200001,57.700001,200.800003,...,8.6,1.2,18.799999,6.8,63.599998,0.4,1.0,1.4,8.35,2.95
1662,131.0,1,127.0,8,22.0,0,21.799999,32.500000,66.900002,226.000000,...,7.3,1.5,20.000000,5.0,46.500000,0.8,0.8,1.5,20.41,4.41
1663,131.0,0,35.0,10,35.0,1,21.799999,32.500000,66.900002,226.000000,...,7.2,0.8,17.600000,5.8,59.200001,0.6,1.4,2.0,6.28,-3.92


In [22]:
#full_dataset['Label'] = full_dataset['Label'].astype('int32')
#label_cols = full_dataset['Date', 'Team']
#full_dataset = full_dataset.drop(['Date', 'Team'], axis=1)
object_columns = full_dataset.select_dtypes(include=['object']).columns
full_dataset[object_columns] = full_dataset[object_columns].astype('float32')
#full_dataset = full_dataset #+ label_cols
full_dataset.dtypes

Team                          float32
Location                        int32
Opponent                      float32
Conference                      int32
Label                         float32
Binary_Label                    int16
pass_completion               float32
pass_attempts                 float32
pass_completion_percent       float32
pass_yards                    float32
pass_td                       float32
rush_attempt                  float32
rush_yards                    float32
avg_rush_yard                 float32
rush_td                       float32
num_plays                     float32
total_yard                    float32
yard_per_play                 float32
pass_first_down               float32
rush_first_down               float32
penalty_first                 float32
first_downs                   float32
penalty                       float32
penalty_yards                 float32
lost_fumble                   float32
interceptions                 float32
turnovers   

In [27]:
full_dataset['Binary_Label']

0       1
1       0
2       1
3       1
4       0
       ..
1660    0
1661    0
1662    0
1663    1
1664    0
Name: Binary_Label, Length: 1665, dtype: int16

In [17]:
save_data(full_dataset, './Dataset/numerical_dataset_srs.pkl')

In [24]:
new_data, scale = normalize_data(full_dataset, './Dataset/MMScaler_srs.pkl', True)

In [25]:
new_data.describe()

,Label,Binary_Label,Team,Location,Opponent,Conference,pass_completion,pass_attempts,pass_completion_percent,pass_yards,...,op_rush_first_down,op_penalty_first,op_first_downs,op_penalty,op_penalty_yards,op_lost_fumble,op_interceptions,op_turnovers,op_SRS,op_SOS
count,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,...,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000,1665.000000
mean,26.902702,0.498498,0.503249,0.303904,0.502712,0.507808,0.524598,0.615495,0.504087,0.437904,...,0.531378,0.381465,0.598011,0.445195,0.442001,0.391892,0.338883,0.453968,0.437902,0.551739
std,13.309108,0.500148,0.290729,0.308848,0.290856,0.331044,0.146887,0.139669,0.137878,0.142588,...,0.169879,0.146234,0.189920,0.148122,0.153595,0.159200,0.125694,0.137482,0.196904,0.223509
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,0.000000,0.250000,0.000000,0.250000,0.200000,0.452830,0.530837,0.416256,0.362330,...,0.431373,0.277778,0.486111,0.338235,0.330065,0.312500,0.240000,0.357143,0.284072,0.339669
50%,27.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.518868,0.614537,0.524631,0.434424,...,0.529412,0.388889,0.618056,0.426471,0.450980,0.375000,0.320000,0.464286,0.454013,0.586745
75%,36.000000,1.000000,0.757576,0.500000,0.757576,0.800000,0.616352,0.700440,0.593596,0.525406,...,0.637255,0.472222,0.722222,0.544118,0.537582,0.500000,0.400000,0.535714,0.565635,0.736355
max,77.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
save_data(new_data, './Dataset/scaled_dataset_test_srs.pkl')

In [ ]:
saved_data = load_data('./Dataset/scaled_dataset.pkl').describe()